In [1]:
library(caret, quiet=TRUE);
library(base64enc)
library(httr, quiet=TRUE)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:
set.seed(1960)

create_model  =  function() {

    model  <- train(Species ~ ., data = iris, method = "ctree")
    
    return(model)
}

In [3]:
# dataset
model = create_model()

In [4]:
# pred <- predict(model, as.matrix(iris[, -5]) , type="prob")
pred_labels <- predict(model, as.matrix(iris[, -5]) , type="raw")
sum(pred_labels != iris$Species)/length(pred_labels)


[1] 0.04

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "caret_rpart_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)

In [7]:
cat(lModelSQL)


WITH "DT_node_lookup" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_2" <= 1.9) THEN 2 ELSE CASE WHEN ("ADS"."Feature_3" <= 1.7) THEN CASE WHEN ("ADS"."Feature_2" <= 4.8) THEN CASE WHEN ("ADS"."Feature_3" <= 1.4) THEN 6 ELSE 7 END ELSE 8 END ELSE CASE WHEN ("ADS"."Feature_2" <= 5.0) THEN 10 ELSE 11 END END END AS node_id_2 
FROM "INPUT_DATA" AS "ADS"), 
"DT_node_data" AS 
(SELECT "Values".nid AS nid, "Values"."P_setosa" AS "P_setosa", "Values"."P_versicolor" AS "P_versicolor", "Values"."P_virginica" AS "P_virginica", "Values"."D" AS "D", "Values"."DP" AS "DP" 
FROM (SELECT 2 AS nid, 1.0 AS "P_setosa", 0.0 AS "P_versicolor", 0.0 AS "P_virginica", 'setosa' AS "D", 1.0 AS "DP" UNION ALL SELECT 6 AS nid, 0.0 AS "P_setosa", 1.0 AS "P_versicolor", 0.0 AS "P_virginica", 'versicolor' AS "D", 1.0 AS "DP" UNION ALL SELECT 7 AS nid, 0.0 AS "P_setosa", 0.9090909090909091 AS "P_versicolor", 0.09090909090909091 AS "P_virginica", 'versicolor' AS "D", 0.9090909090909091 AS "DP" UNION ALL 

# Execute the SQL Code

In [8]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [9]:
dataset = iris[,-5]

df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

Feature_0,Feature_1,Feature_2,Feature_3,KEY
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,2
4.7,3.2,1.3,0.2,3
4.6,3.1,1.5,0.2,4
5.0,3.6,1.4,0.2,5
5.4,3.9,1.7,0.4,6


In [10]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [11]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

KEY,Score_setosa,Score_versicolor,Score_virginica,Proba_setosa,Proba_versicolor,Proba_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
1,NA,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
2,NA,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
3,NA,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
4,NA,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
5,NA,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
6,NA,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1


# R Caret Rpart Output

In [12]:
# pred_proba  =  predict(model, as.matrix(iris[,-5]), type = "prob")
df_r_out = data.frame(seq.int(nrow(dataset))) # (pred_proba)
names(df_r_out) = c("KEY") # sprintf("Proba_%s",model$levels)

df_r_out$KEY = seq.int(nrow(dataset))
df_r_out$Score_setosa  =  NA
df_r_out$Score_versicolor  =  NA
df_r_out$Score_virginica  =  NA
df_r_out$Proba_setosa  =  NA
df_r_out$Proba_versicolor  =  NA
df_r_out$Proba_virginica  =  NA
df_r_out$LogProba_setosa  =  log(df_r_out$Proba_setosa)
df_r_out$LogProba_versicolor =  log(df_r_out$Proba_versicolor)
df_r_out$LogProba_virginica  =  log(df_r_out$Proba_virginica)
df_r_out$Decision =   predict(model, as.matrix(iris[,-5]), type = "raw")
df_r_out$DecisionProba =  NA
head(df_r_out)



KEY,Score_setosa,Score_versicolor,Score_virginica,Proba_setosa,Proba_versicolor,Proba_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,setosa,NA
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,setosa,NA
3,NA,NA,NA,NA,NA,NA,NA,NA,NA,setosa,NA
4,NA,NA,NA,NA,NA,NA,NA,NA,NA,setosa,NA
5,NA,NA,NA,NA,NA,NA,NA,NA,NA,setosa,NA
6,NA,NA,NA,NA,NA,NA,NA,NA,NA,setosa,NA


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Score_setosa_1,Score_versicolor_1,Score_virginica_1,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
3,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
4,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
5,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1
6,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,0,0,0,-1.797693e+307,-1.797693e+307,setosa,1


In [14]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”

KEY,Score_setosa_1,Score_versicolor_1,Score_virginica_1,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2


In [15]:
stopifnot(nrow(diffs_df) == 0)

In [16]:
summary(df_r_out)

      KEY         Score_setosa   Score_versicolor Score_virginica
 Min.   :  1.00   Mode:logical   Mode:logical     Mode:logical   
 1st Qu.: 38.25   NA's:150       NA's:150         NA's:150       
 Median : 75.50                                                  
 Mean   : 75.50                                                  
 3rd Qu.:112.75                                                  
 Max.   :150.00                                                  
                                                                 
 Proba_setosa   Proba_versicolor Proba_virginica LogProba_setosa
 Mode:logical   Mode:logical     Mode:logical    Min.   : NA    
 NA's:150       NA's:150         NA's:150        1st Qu.: NA    
                                                 Median : NA    
                                                 Mean   :NaN    
                                                 3rd Qu.: NA    
                                                 Max.   : NA    
                 

In [17]:
summary(df_sql_out)

      KEY         Score_setosa   Score_versicolor Score_virginica
 Min.   :  1.00   Mode:logical   Mode:logical     Mode:logical   
 1st Qu.: 38.25   NA's:150       NA's:150         NA's:150       
 Median : 75.50                                                  
 Mean   : 75.50                                                  
 3rd Qu.:112.75                                                  
 Max.   :150.00                                                  
  Proba_setosa    Proba_versicolor Proba_virginica  LogProba_setosa      
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :-1.798e+307  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:-1.798e+307  
 Median :0.0000   Median :0.0000   Median :0.0000   Median :-1.798e+307  
 Mean   :0.3333   Mean   :0.3333   Mean   :0.3333   Mean   :-1.198e+307  
 3rd Qu.:1.0000   3rd Qu.:0.9091   3rd Qu.:0.9688   3rd Qu.:  0.000e+00  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :  0.000e+00  
 LogProba_versicolor